### Prueba 2: usar edades numéricas escaladas completas de víctima y llamante


In [1]:
import numpy as np
import os
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler, OneHotEncoder
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, classification_report
import gc

In [2]:
dataset_dir = os.path.join(os.path.dirname(os.path.dirname(os.path.abspath(''))), 'datasets')

In [15]:
llamados_v5= pd.read_excel(os.path.join(dataset_dir, 'xlsx/llamados_v5.xlsx'), parse_dates=['llamado_fecha_hora'])


In [16]:
llamados_v5.loc[:, 'victima_convive_agresor'] = llamados_v5['victima_convive_agresor'].replace({'NS/NC': pd.NA})
llamados_v5.drop('agresor_fam_no_fam', axis=1, inplace=True)

scaler = MinMaxScaler()
encoder = OrdinalEncoder()


llamados_v5['timestamp_encoded'] = encoder.fit_transform(llamados_v5[['llamado_fecha_hora']])
llamados_v5['timestamp_encoded_scaled'] = scaler.fit_transform(llamados_v5[['timestamp_encoded']])
    

llamados_v5['victima_edad_escalada'] = scaler.fit_transform(llamados_v5[['victima_edad']])
llamados_v5['llamante_edad_escalada'] = scaler.fit_transform(llamados_v5[['llamante_edad']])

llamados_v5['momento_encoded'] = encoder.fit_transform(llamados_v5[['momento_dia']])
llamados_v5['estacion_encoded'] = encoder.fit_transform(llamados_v5[['estacion_del_año']])

llamados_v5['momento_encoded_scaled'] = scaler.fit_transform(llamados_v5[['momento_encoded']])
llamados_v5['estacion_encoded_scaled'] = scaler.fit_transform(llamados_v5[['estacion_encoded']])

In [17]:
# Find columns that only contain 'SI' and 'NO'

si_no_columns = []
cat_mas_de_dos = []
for column in llamados_v5.columns:
    if llamados_v5[column].dtypes == object:
        if set(llamados_v5[column].unique()).issubset({'SI', 'NO'}):
            si_no_columns.append(column)
        else:
            cat_mas_de_dos.append(column)

# encodear si/no 

for col in si_no_columns:
    llamados_v5[col] = llamados_v5[col].map({'SI': 1, 'NO':0})


#encodear target
llamados_v5['victima_convive_agresor'] = llamados_v5['victima_convive_agresor'].map({'SI': 1, 'NO':0})

In [18]:
# droppear las columnas que ya sé que no voy a usar
llamados_v5.drop([ 'momento_encoded', 'estacion_encoded', 'momento_dia',
                 'estacion_del_año', 'victima_edad', 'llamante_edad',  
                 'llamado_fecha_hora', 'timestamp_encoded'], axis=1, inplace=True) 

In [19]:
encodear_one_hot = ['llamante_genero',
 'caso_judicializado',
 'victima_genero',
 'victima_nacionalidad',
 'victima_discapacidad',
 'genero_agresor',
 'agresor_conocido_no_conocido',
 'tipo_vinculo_llamante',
 'hecho_lugar_red',
 'llamado_provincia_red']

In [20]:
# encodear con one hot todas las categóricas de más de dos valores: encodear_one_hot

encoder = OneHotEncoder(sparse_output=True) # considerar drop first is sparse false

# Fit and transform the data
encoded_sparse = encoder.fit_transform(llamados_v5[encodear_one_hot])

# Convert the sparse matrix to a dense array
encoded_array = encoded_sparse.toarray()

# Create a DataFrame with the encoded columns
encoded_df = pd.DataFrame(encoded_array, columns=encoder.get_feature_names_out(encodear_one_hot))

# Drop the original columns that were one-hot encoded
llamados_v5.drop(columns=encodear_one_hot, inplace=True)

# Concatenate the original DataFrame with the encoded DataFrame
llamados_v5 = pd.concat([llamados_v5, encoded_df], axis=1)

In [21]:
# sacar todos los casos vacíos de edad

llamados_v5.dropna(subset=['victima_edad_escalada'], inplace=True)
llamados_v5.dropna(subset=['llamante_edad_escalada'], inplace=True)

In [22]:
# sacar todos los casos vacíos de convive y ponerlos en un nuevo dataset aparte, 
# será el test final a predecir con el mejor modelo

# Create a DataFrame with rows where 'convive' is NA
test_final = llamados_v5[llamados_v5['victima_convive_agresor'].isna()]

# Remove rows where 'convive' is NA from the original DataFrame
llamados_v5 = llamados_v5.dropna(subset=['victima_convive_agresor'])

In [23]:
llamados_v5.isnull().values.any()

False

In [24]:
gc.collect()

108

In [25]:
# Separate features and target variable
X = llamados_v5.drop(columns=['victima_convive_agresor'])
y = llamados_v5['victima_convive_agresor']

# Stratified split
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in sss.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

# Define the parameter grid for GridSearchCV
param_grid = {
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 'auto']
}

# Initialize the SVM model
svm = SVC()

# Perform GridSearchCV to find the best parameters
grid_search = GridSearchCV(svm, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Get the best model
best_model = grid_search.best_estimator_

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Evaluate the model
print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation accuracy: ", grid_search.best_score_)
print("Test set accuracy: ", accuracy_score(y_test, y_pred))
print("Classification report:\n", classification_report(y_test, y_pred))

Best parameters found:  {'C': 1, 'gamma': 'scale', 'kernel': 'poly'}
Best cross-validation accuracy:  0.8817828497604564
Test set accuracy:  0.8881431767337807
Classification report:
               precision    recall  f1-score   support

         0.0       0.90      0.97      0.93      1472
         1.0       0.77      0.52      0.62       316

    accuracy                           0.89      1788
   macro avg       0.84      0.74      0.78      1788
weighted avg       0.88      0.89      0.88      1788



In [26]:
# Apply the best model to the subset of the original dataset
X_na = test_final.drop(columns=['victima_convive_agresor'])
na_predictions = best_model.predict(X_na)

# Add the predictions to the na_convive_df DataFrame
test_final['victima_convive_agresor_pred'] = na_predictions

print("\nPredictions for NA 'convive' values:")
print(test_final)


Predictions for NA 'convive' values:
       victima_a_resguardo  victima_convive_agresor  vs_tocamiento_sexual  \
14                       1                      NaN                     0   
41                       1                      NaN                     0   
54                       1                      NaN                     0   
55                       1                      NaN                     0   
58                       1                      NaN                     0   
...                    ...                      ...                   ...   
19097                    1                      NaN                     0   
19108                    1                      NaN                     0   
19113                    1                      NaN                     0   
19121                    1                      NaN                     0   
19129                    1                      NaN                     0   

       vs_intento_tocamiento  vs_groo

In [27]:
test_final.victima_convive_agresor_pred.value_counts()


victima_convive_agresor_pred
0.0    1182
1.0     133
Name: count, dtype: int64

In [28]:
test_final.shape


(1315, 76)

In [29]:
llamados_v5.shape


(8936, 75)

In [30]:
llamados_v5.victima_convive_agresor.value_counts()

victima_convive_agresor
0.0    7355
1.0    1581
Name: count, dtype: int64

In [32]:
print('Proporción de SI en dataset original', llamados_v5.victima_convive_agresor.value_counts()[1]/llamados_v5.shape[0], 'Proporción de SI en test final',test_final.victima_convive_agresor_pred.value_counts()[1]/test_final.shape[0] )
print('Proporción de NO en dataset original', test_final.victima_convive_agresor_pred.value_counts()[1]/test_final.shape[0], 'Proporción de NO en test final', test_final.victima_convive_agresor_pred.value_counts()[0]/test_final.shape[0])


Proporción de SI en dataset original 0.17692479856759177 Proporción de SI en test final 0.10114068441064639
Proporción de NO en dataset original 0.8230752014324082 Proporción de NO en test final 0.8988593155893536
